In [116]:
import sys
import os                           # for os interface
import csv
import math
import pickle                       # for full-precision data storage
import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict  # append list to disc
import timeit

In [117]:
start_time = timeit.default_timer()
data_file = '../input/itcont.txt'
#test_data = '../input/test_data.txt'
header_file = '../input/indiv_header_file.csv'
elapsed = timeit.default_timer() - start_time
print (elapsed)

6.367999594658613e-05


In [118]:
start_time = timeit.default_timer()
# read header line
indiv_header=[]
with open(header_file) as f:
    for line in f:
        header_ele=[ele.strip() for ele in line.split(',') ]
        indiv_header.extend(header_ele) # use extend instead of append
elapsed = timeit.default_timer() - start_time
print (elapsed)
dataset_raw = pd.read_csv(data_file,delimiter='|',\
                      names=indiv_header,keep_default_na=False,dtype=str)
elapsed = timeit.default_timer() - start_time
print (elapsed)

0.0015671770088374615
0.06247531500412151


In [123]:
start_time = timeit.default_timer()
# keep columns of interest
dataset_valid=pd.DataFrame()
dataset_valid=dataset_raw.drop(['AMNDT_IND', 'RPT_TP','TRANSACTION_PGI',\
              'IMAGE_NUM','TRANSACTION_TP','ENTITY_TP',\
              'CITY','STATE','EMPLOYER','OCCUPATION','TRAN_ID',
              'FILE_NUM','MEMO_CD','MEMO_TEXT','SUB_ID'], axis=1)

dataset_valid=dataset_valid[dataset_valid['OTHER_ID'].notnull()]
#dataset_valid=dataset_valid[dataset_valid.notnull()]
dataset_valid=dataset_valid.dropna(how='any')
elapsed = timeit.default_timer() - start_time
print (elapsed)

0.009852956995018758


In [124]:
start_time = timeit.default_timer()
# take the first five characters of 'ZIP_CODE'
# dataset_valid['ZIP_CODE']=dataset_valid['ZIP_CODE'].astype(str).str[0:5]
# 'ZIP_CODE' is valid only when the resulting string length is 5
dataset_valid=dataset_valid[dataset_valid['ZIP_CODE'].str.len()>=5]

# take integer in 'TRANSACTION_AMT'
dataset_valid['TRANSACTION_AMT']=dataset_valid['TRANSACTION_AMT'].astype(int)
# 'TRANSACTION_AMT' is valid only when > 0
dataset_valid=dataset_valid[dataset_valid['TRANSACTION_AMT']>0]

In [125]:
# Keep all duplicates
dataset_valid=dataset_valid[dataset_valid.duplicated\
                             (['NAME','ZIP_CODE'],keep=False)]

In [70]:
dataset_valid.iloc[35:55]

,CMTE_ID,NAME,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID
295,C00415026,"THOMPSON, CHET",20006,01132017,208,
296,C00415026,"THOMPSON, CHET",20006,01312017,208,
315,C00364778,"SAGGURTI, PURNA R",10036,01132017,208,
316,C00364778,"BANKS, KEITH T",10036,01132017,210,
317,C00364778,"BANKS, KEITH T",10036,01312017,210,
319,C00364778,"SAGGURTI, PURNA R",10036,01312017,208,
322,C00364778,"BANKS, KEITH T",10036,01112017,40,
492,C00427930,THE CHICKASAW NATION,74820,01312017,2500,
496,C00365502,"CANAFAX, JAMES D.",282773613,01112017,208,
497,C00365502,"FEES, JOHN A",245036496,01142017,208,


In [130]:
# date order
#donate_time=pd.to_datetime(dataset_valid['TRANSACTION_DT'], format='%m%d%Y', errors='ignore')
#dataset_valid['newcol1'] = donate_time.shift() - donate_time
#dataset_valid['newcol1'] = dataset_valid['newcol1'].fillna(0)
#dataset_valid['newcol2'] = pd.to_timedelta(dataset_valid['newcol1'],errors='coerce').astype(str).str[0:2]

# identity=NAME+ZIP
dataset_repeat=pd.DataFrame()
dataset_repeat['Identity'] = dataset_valid['NAME'] + dataset_valid.ZIP_CODE.str[0:5]
dataset_repeat['CMTE_ID'] = dataset_valid['CMTE_ID']
dataset_repeat['TRANS_AMT'] = dataset_valid['TRANSACTION_AMT']
#dataset_repeat['index1'] = dataset_valid.index
dataset_repeat['date'] = pd.to_datetime(dataset_valid['TRANSACTION_DT'], format='%m%d%Y', errors='ignore')

#dataset_year.sort_values(by=['Identity','index1'])

In [131]:
grouped=dataset_repeat.groupby('Identity')
# Shift the previous data from the same group and take the difference
dataset_repeat['date_shifted'] = dataset_repeat.groupby(['Identity'])['date'].shift(1)
dataset_repeat['date_diff'] = dataset_repeat['date_shifted'] - dataset_repeat['date']
#dataset_repeat['date_diff'] = pd.to_timedelta(dataset_repeat['date_diff']).astype(str).str[0:2]

In [134]:
dataset_repeat[120:150]

,Identity,CMTE_ID,TRANS_AMT,date,date_shifted,date_diff
646,"NAYLOR, JOSEPH M94583",C00035006,208,2017-01-20,NaT,NaT
647,"MOSHIRI, ALIREZA94583",C00035006,208,2017-01-06,NaT,NaT
648,"MOSHIRI, ALIREZA94583",C00035006,208,2017-01-20,2017-01-06,-14 days
649,"NAYLOR, JOSEPH M94583",C00035006,208,2017-01-06,2017-01-20,14 days
650,"NEFF, ROBERT C JR94583",C00035006,208,2017-01-06,NaT,NaT
651,"OLSON, STACEY G94583",C00035006,208,2017-01-06,NaT,NaT
652,"NEFF, ROBERT C JR94583",C00035006,208,2017-01-20,2017-01-06,-14 days
653,"NELSON, MARK A94583",C00035006,208,2017-01-06,NaT,NaT
654,"NELSON, MARK A94583",C00035006,208,2017-01-20,2017-01-06,-14 days
655,"OLSON, STACEY G94583",C00035006,208,2017-01-20,2017-01-06,-14 days


In [111]:
# drop the first occurence
dataset_repeat=dataset_repeat.dropna(how='any')
# drop the out of order (positives 'date_diff') records
dataset_repeat=dataset_repeat[dataset_repeat['date_diff'].astype(int)<=0]


#pd.to_datetime(dataset_year['date_shifted'], format='%m%d%Y', errors='ignore')
#for group in grouped:
    # transforming to tuples so that it can be used as keys on a dict
#    lines = [tuple(y) for y in group[1].loc[:,'newcol2'].values.tolist()] 
#    group[group.columns.difference(['newcol2'])]
    #key = tuple(lines) 
    
    
#dataset_valid=dataset_valid[dataset_valid['newcol2'].astype(int)<0]
#grouped=dataset_arrange.groupby('Identity')
#dataset_valid.groupby('NAME')
dataset_repeat.iloc[0]

Identity        CAFARO, CAPRI S44425
CMTE_ID                    C00394973
TRANS_AMT                        396
date             2017-01-10 00:00:00
date_shifted     2017-01-06 00:00:00
date_diff          -4 days +00:00:00
Name: 109, dtype: object

In [114]:
start_time = timeit.default_timer()
dataset_arrange=pd.DataFrame()
dataset_arrange['Identity'] = dataset_repeat['CMTE_ID'] + dataset_repeat['Identity'].str[-5:] + dataset_repeat['date'].dt.year.astype(str)
dataset_arrange['TRANS_AMT'] = dataset_repeat['TRANS_AMT']
#dataset_arrange['index1'] = dataset_arrange.index
elapsed = timeit.default_timer() - start_time
print (elapsed)

0.006262826995225623


In [115]:
dataset_arrange.iloc[10:25]

,Identity,TRANS_AMT
228,C00161414452452017,208
249,C00275529782662017,536
264,C00035691802022017,205
267,C00035691770722017,208
269,C00035691770722017,208
271,C00035691770722017,208
275,C00035691770322017,208
280,C00035691770422017,190
296,C00415026200062017,208
317,C00364778100362017,210


In [71]:

grouped=dataset_arrange.groupby('Identity')
#grouped.groups['C00394973444252017']
#pd.rolling_quantile(grouped['TRANS_AMT'], 2, 0.3)
grouped['TRANS_AMT'].expanding(min_periods=1,center=False).quantile(quantile=0.3,axis=0,numeric_only=False,interpolation='nearest')
#grouped['TRANS_AMT'].expanding(1).quantile(quantile=0.3,interpolation='nearest')
#total=grouped['TRANS_AMT'].expanding(1).sum()
#times=grouped['TRANS_AMT'].expanding(1).count()
#def percentile(group):
#    s1=group.expanding(1)
#    return np.percentile(s1, 50)
def f1(group):
    return pd.DataFrame({'Perc': group['TRANS_AMT'].expanding(min_periods=1, center=False).quantile(quantile=0.3)})


def f2(group):
    return pd.DataFrame({'Perc': group['TRANS_AMT'].expanding(min_periods=1, center=False)})

#Results = grouped.apply(f)

def rank(array):
    s = pd.Series(array)
    return s.rank(ascending=False)[len(s)-1]

#grouped['TRANS_AMT'].apply(dataset_arrange['TRANS_AMT'], 3, rank)
pd.rolling_apply(grouped['TRANS_AMT'], 3, rank)
#grouped.['TRANS_AMT'].expanding(min_periods=1,center=False).apply
#s1=grouped['TRANS_AMT'].expanding(min_periods=1,center=False)

#s1.series.expand()
#def func_sum(group):
#    return pd.DataFrame({'sum': group.expanding(1).sum(),'times':group.expanding(1).count()} )
#def func_perc(group):
#    return pd.Series(group.expanding(min_periods=1).apply(func=np.percentile(a= q=30)))

#a.rolling(row).apply(func=np.percentile, args=(25,)).tail(1)
#def func_total(group):
#    return pd.Series(group.expanding(1).sum())
#def func_times(group):
#    return pd.Series(group.expanding(1).count())

#grouped['TRANS_AMT'].transform(func_perc)



/Users/YueQian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: pd.rolling_apply is deprecated for SeriesGroupBy and will be removed in a future version, replace with 
	SeriesGroupBy.rolling(window=3,center=False).apply(func=<function>,args=<tuple>,kwargs=<dict>)


Identity               
C00033555317922017  519    NaN
C00035006945832017  615    NaN
                    618    NaN
                    620    2.0
                    622    2.0
                    624    2.0
                    626    2.0
                    628    2.0
                    630    2.0
                    632    2.0
                    634    2.0
                    636    2.0
                    640    2.0
                    641    2.0
                    643    2.0
                    645    2.0
                    648    2.0
                    649    2.0
                    652    2.0
                    654    2.0
                    655    2.0
                    657    2.0
                    660    2.0
                    662    2.0
                    666    2.0
                    667    2.0
                    668    2.0
                    673    2.0
                    674    2.0
                    676    2.0
                          ... 
C00193433112182

In [48]:
#grouped=dataset_arrange.groupby('Identity')['TRANS_AMT']
#grouped.expanding(min_periods=1).quantile(quantile=0.3,interpolation='nearest')

In [58]:
X

,Perc
109,396.0
110,136.3
124,190.0
208,500.0
215,125.0
217,174.0
219,223.0
221,375.0
223,213.2
225,450.0


In [152]:
start_time = timeit.default_timer()

dd = defaultdict(list)
dataset_final = dataset_arrange.to_dict('records', into=dd)

d1 = defaultdict(list)
d2 = defaultdict(list)
perc = 30.
percent = perc/100.



for item in dataset_final:
    identity=item['Identity']
    amount=item['TRANS_AMT']
    index1=item['index1']
    d1[identity].append(amount)
    if len(d1[identity]) ==1:
        perc_of_cont = amount
        total_cont = amount
        time_of_cont =1
        continue
    else:
        ranked = sorted(d1[identity])
        n = int(round(percent * len(d1[identity])))
        perc_of_cont=ranked[n-1]
        total_cont=sum(d1[identity])
        time_of_cont=len(d1[identity])
    

    d2[index1].append(identity)
    d2[index1].append(perc_of_cont)
    d2[index1].append(total_cont)
    d2[index1].append(time_of_cont)    
    
elapsed = timeit.default_timer() - start_time
print (elapsed)

    
#for k, v in d1.items():       
#    print (k,v)

944.451614868005


In [153]:
#%timeit dataset_final.items()

In [154]:
# group by and sum
# https://stackoverflow.com/questions/33823091/python-pandas-counting-the-frequency-of-a-specific-value-in-each-row-of-datafra
# https://stackoverflow.com/questions/42446006/pandas-find-previous-row-of-matching-value
#data_output['TRANS_AMT_PRE'] = \
#data_output.groupby(['CMTE_ID','ZIP_CODE','TRANS_DT'])['TRANS_AMT'].shift()
#data_output['TRANS_AMT'] + \
#data_output.groupby('ZIP_CODE')['TRANS_AMT'].shift()

In [155]:
#grouped=data_output.groupby(['CMTE_ID','ZIP_CODE','TRANS_DT'])

In [156]:
#new=data_output.groupby(['CMTE_ID']).get_group('C00035006')
#grouped.agg({'TRANS_AMT':np.sum,'TRANS_AMT': np.mean})

In [157]:
#for name, group in grouped:
    #print(name)
#    print(group.TRANS_AMT)

In [158]:
#grouped.bfill